In [3]:
using PCT
using Revise

Precompiling PCT
  ✓ PCT
  1 dependency successfully precompiled in 1 seconds. 7 already precompiled.


## Differentiating through linear ops (Adjoint method)

In [4]:
Res, _ = @pct (g::RF, w::RF) -> begin
    let conv = (f::RF) -> (t::R) -> ∑((τ::R), f(τ) * g(t - τ))
        pullback((f::RF) -> ∑((t::R), w(t) * conv(f)(t))
    )
    end
end; Res

(g, w) -> 
    let 
        conv = (f) -> 
            (t) -> 
                ∑((τ), f(τ)⋅g((t-1.0⋅τ)))
        𝒫((f) -> 
            ∑((t), w(t)⋅conv(f)(t)))
    end

In [5]:
vdiff(eval_all(Res))

1 contract_delta
∑((t, τ), δ(τ, _d, w(t)⋅g((t-1.0⋅τ))))
Sum
∑((t), w(t)⋅g((t-1.0⋅_d)))
-->



(g, w) -> 
    (f) -> 
        (_d) -> 
            ∑((t), w(t)⋅g((t-1.0⋅_d)))

Conv and Cross-correlation are adjoints.

In [6]:
conv, _ = @pct (g::RF) -> pullback((f::RF) -> (t::R) -> ∑((τ::R), f(τ) * g(t - τ))); conv

(g) -> 
    𝒫((f) -> 
        (t) -> 
            ∑((τ), f(τ)⋅g((t-1.0⋅τ))))

In [17]:
reduce_pullback(conv)

1 contract_delta
∑((t, τ), δ(τ, _d, _k(t)⋅g((t-1.0⋅τ))))
Sum
∑((t), _k(t)⋅g((t-1.0⋅_d)))
-->



(g) -> 
    (f, _k) -> 
        (_d) -> 
            ∑((t), _k(t)⋅g((t-1.0⋅_d)))

## Deriving the Conjugate Gradient

In [22]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) - sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::N) -> x(i) + α * (r(i) + β * p(i))))
end; blaserize(f)

1 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i, j), 0.5⋅x(i)⋅x(j)⋅A(i, j))-1.0⋅∑((i), x(i)⋅b(i)))
        𝒫((α, β) -> 
            E((i) -> 
                ((β⋅p(i)+r(i))⋅α+x(i))))
    end
Map
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((j), 0.5⋅(xᵀ⋅A)(j)⋅x(j))-1.0⋅∑((i), x(i)⋅b(i)))
        𝒫((α, β) -> 
            E((i) -> 
                ((β⋅p(i)+r(i))⋅α+x(i))))
    end
-->

2 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((j), 0.5⋅(xᵀ⋅A)(j)⋅x(j))-1.0⋅∑((i), x(i)⋅b(i)))
        𝒫((α, β) -> 
            E((i) -> 
                ((β⋅p(i)+r(i))⋅α+x(i))))
    end
Map
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (-1.0⋅∑((i), x(i)⋅b(i))+0.5⋅xᵀ⋅A⋅x)
        𝒫((α, β) -> 
            E((i) -> 
                ((β⋅p(i)+r(i))⋅α+x(i))))
    end
-->

3 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (-1.0⋅∑((i), x(i)⋅b(i))+0.5⋅xᵀ⋅A⋅x)
        𝒫((α, β) -> 
            E((i) -> 
  

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (-1.0⋅xᵀ⋅b+0.5⋅xᵀ⋅A⋅x)
        𝒫((α, β) -> 
            E((α⋅(β⋅p+r)+x)))
    end

In [23]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

1 swallow
(∑((_z, i), -1.0⋅(β⋅p(i)+r(i))⋅b(i)⋅δ(∑((i, j), ((β⋅p(i)+r(i))⋅α+x(i))⋅((β⋅p(j)+r(j))⋅α+x(j))⋅0.5⋅A(i, j)), _z, 1))+∑((i, j), (∑((_z_1), (β⋅p(j)+r(j))⋅_z_1⋅0.5⋅A(i, j)⋅δ(((β⋅p(i)+r(i))⋅α+x(i)), _z_1, 1))+((β⋅p(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅0.5⋅A(i, j))))
Add
(∑((_z, i), δ(_z, ∑((i, j), ((β⋅p(i)+r(i))⋅α+x(i))⋅((β⋅p(j)+r(j))⋅α+x(j))⋅0.5⋅A(i, j)), -1.0⋅(β⋅p(i)+r(i))⋅b(i)))+∑((i, j), (∑((_z_1), (β⋅p(j)+r(j))⋅_z_1⋅0.5⋅A(i, j)⋅δ(((β⋅p(i)+r(i))⋅α+x(i)), _z_1, 1))+((β⋅p(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅0.5⋅A(i, j))))
-->

2 contract_delta
(∑((_z, i), δ(_z, ∑((i, j), ((β⋅p(i)+r(i))⋅α+x(i))⋅((β⋅p(j)+r(j))⋅α+x(j))⋅0.5⋅A(i, j)), -1.0⋅(β⋅p(i)+r(i))⋅b(i)))+∑((i, j), (∑((_z_1), (β⋅p(j)+r(j))⋅_z_1⋅0.5⋅A(i, j)⋅δ(((β⋅p(i)+r(i))⋅α+x(i)), _z_1, 1))+((β⋅p(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅0.5⋅A(i, j))))
Add
(∑((i), -1.0⋅(β⋅p(i)+r(i))⋅b(i))+∑((i, j), (∑((_z_1), (β⋅p(j)+r(j))⋅_z_1⋅0.5⋅A(i, j)⋅δ(((β⋅p(i)+r(i))⋅α+x(i)), _z_1, 1))+((β⋅p(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅0.5⋅A(i, j))))
-->

3 sum_dist
(∑((i),

4 gcd
(∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅A(j, i))-1.0⋅∑((i), (β⋅p(i)+r(i))⋅b(i))), (∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅p(i)⋅A(i, j))⋅α-1.0⋅∑((i), b(i)⋅p(i))⋅α)
PCTVector
(∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅A(j, i))-1.0⋅∑((i), (β⋅p(i)+r(i))⋅b(i))), (∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅p(i)⋅A(i, j))-1.0⋅∑((i), b(i)⋅p(i)))⋅α
-->

1 scalar tensor product
(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅A(j, i))-1.0⋅∑((i), (β⋅p(i)+r(i))⋅b(i))), (∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅p(i)⋅A(i, j))-1.0⋅∑((i), b(i)⋅p(i)))⋅α
Map
(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i, j), (((β⋅p)(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅A(j, i))-1.0⋅∑((i), (β⋅p(i)+r(i))⋅b(i))), (∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅p(i)⋅A(i, j))-1.0⋅∑((i), b(i)⋅p(i)))⋅α
-->

2 tensor addition
(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i, j), (((β⋅p)(j)+r(j))⋅α+x(j))⋅(β⋅p(i)+r(i))⋅A(j, i))-1.0⋅∑((i), (β⋅p(i)+r(i))⋅b(i))), (∑((i, j), ((β⋅p(j)+r(j))⋅α+x(j))⋅p(i)⋅A(i, j))-1.0

(A, r, p, b, x) -> 
    (α, β) -> 
        (-1.0⋅(β⋅p+r)ᵀ⋅b+(α⋅(β⋅p+r)+x)ᵀ⋅A⋅(β⋅p+r)), (-1.0⋅bᵀ⋅p+pᵀ⋅A⋅(α⋅(β⋅p+r)+x))⋅α

## Nonlinear/Tensor Algebra

In [27]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j)));  blaserize(f)

1 conjugate call
(A) -> 
    𝒫((x) -> 
        ∑((i, j), x(i)'⋅x(j)⋅A(i, j)))
Map
(A) -> 
    𝒫((x) -> 
        ∑((i, j), x(j)⋅A(i, j)⋅x⁺(i)))
-->

2 blas mul
(A) -> 
    𝒫((x) -> 
        ∑((i, j), x(j)⋅A(i, j)⋅x⁺(i)))
Map
(A) -> 
    𝒫((x) -> 
        ∑((j), (x⁺⋅A)(j)⋅x(j)))
-->

3 blas mul
(A) -> 
    𝒫((x) -> 
        ∑((j), (x⁺⋅A)(j)⋅x(j)))
Map
(A) -> 
    𝒫((x) -> 
        x⁺⋅A⋅x)
-->



(A) -> 
    𝒫((x) -> 
        x⁺⋅A⋅x)

In [28]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

1 sum_dist
∑((i, j), ((_d) -> 
    ∑((_z_1), δ(j, _d, _z_1'⋅A(i, j)'⋅δ(x(i)', _z_1, 1)))+(_d) -> 
    δ(i, _d, x(j)⋅A(i, j))))
Add
((_d) -> 
    ∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+(_d) -> 
    ∑((i, j, _z_1), δ(j, _d, _z_1'⋅A(i, j)'⋅δ(x(i)', _z_1, 1))))
-->

2 combine map
((_d) -> 
    ∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+(_d) -> 
    ∑((i, j, _z_1), δ(j, _d, _z_1'⋅A(i, j)'⋅δ(x(i)', _z_1, 1))))
Map
(_d) -> 
    (∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+∑((i, j, _z_1), δ(j, _d, _z_1'⋅A(i, j)'⋅δ(x(i)', _z_1, 1))))
-->

3 contract_delta
(_d) -> 
    (∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+∑((i, j, _z_1), δ(j, _d, _z_1'⋅A(i, j)'⋅δ(x(i)', _z_1, 1))))
Map
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑((i, j, _z_1), δ(j, _d, _z_1'⋅A(i, j)'⋅δ(x(i)', _z_1, 1))))
-->

4 contract_delta
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑((i, j, _z_1), δ(j, _d, _z_1'⋅A(i, j)'⋅δ(x(i)', _z_1, 1))))
Map
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑((_z_1, i), _z_1'⋅A(i, _d)'⋅δ(x(i)', _z_1, 1)))
-->

5 swallow
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑(

(A) -> 
    (x) -> 
        2.0⋅A⋅x

In [44]:
_, ctx = @pct begin
    @space SS begin
        type=(N,N,N) -> R
        symmetries=(((2,1,3), :id), ((1,3,2), :id))
    end
end; 

In [45]:
f, _ = @pct _ ctx (T::SS) -> pullback((x::RV) -> sum((i, j, k), T(i, j, k) * x(i) * x(j) * x(k))); f

(T) -> 
    𝒫((x) -> 
        ∑((i, j, k), x(i)⋅x(j)⋅x(k)⋅T(i, j, k)))

In [49]:
redux(vdiff(f); settings=symmetry_settings)

1 sum_dist
∑((i, j, k), (∑((_z_1), ((_d) -> 
    ∑((_z_2), δ(k, _d, _z_2⋅_z_1⋅T(i, j, k)⋅δ(x(j), _z_2, δ(x(i), _z_1, 1))))+(_d) -> 
    δ(j, _d, _z_1⋅x(k)⋅T(i, j, k)⋅δ(x(i), _z_1, 1))))+(_d) -> 
    δ(i, _d, x(j)⋅x(k)⋅T(i, j, k))))
Add
(∑((i, j, k, _z_1), ((_d) -> 
    ∑((_z_2), δ(k, _d, _z_2⋅_z_1⋅T(i, j, k)⋅δ(x(j), _z_2, δ(x(i), _z_1, 1))))+(_d) -> 
    δ(j, _d, _z_1⋅x(k)⋅T(i, j, k)⋅δ(x(i), _z_1, 1))))+(_d) -> 
    ∑((i, j, k), δ(i, _d, x(j)⋅x(k)⋅T(i, j, k))))
-->

2 sum_dist
(∑((i, j, k, _z_1), ((_d) -> 
    ∑((_z_2), δ(k, _d, _z_2⋅_z_1⋅T(i, j, k)⋅δ(x(j), _z_2, δ(x(i), _z_1, 1))))+(_d) -> 
    δ(j, _d, _z_1⋅x(k)⋅T(i, j, k)⋅δ(x(i), _z_1, 1))))+(_d) -> 
    ∑((i, j, k), δ(i, _d, x(j)⋅x(k)⋅T(i, j, k))))
Add
((_d) -> 
    ∑((i, j, k), δ(i, _d, x(j)⋅x(k)⋅T(i, j, k)))+(_d) -> 
    ∑((i, j, k, _z_1), δ(j, _d, _z_1⋅x(k)⋅T(i, j, k)⋅δ(x(i), _z_1, 1)))+(_d) -> 
    ∑((i, j, k, _z_1, _z_2), δ(k, _d, _z_2⋅_z_1⋅T(i, j, k)⋅δ(x(j), _z_2, δ(x(i), _z_1, 1)))))
-->

3 combine map
((_d) -> 
    ∑((i, j,

(T) -> 
    (x) -> 
        (_d) -> 
            ∑((i, k), x(i)⋅x(k)⋅T(i, _d, k))⋅3.0